In [ ]:
import cv2 
from pynq import Overlay 
from pynq import allocate 
from pynq.lib.video import *
import numpy as np
from pynq.lib import AxiGPIO

ol = Overlay("./HDMI_720.bit")

vdma = ol.axi_vdma_0
frame_in_w = 1280
frame_in_h = 720


mode = VideoMode(frame_in_w, frame_in_h,24)
vdma.writechannel.mode = mode
vdma.writechannel.start()

# camera (input) configuration
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("無法開啟攝影機")
    exit()
else:
    print("攝影機成功開啟")
    
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);


buttons_instance = ol.ip_dict["axi_gpio_0"]
buttons = AxiGPIO(buttons_instance).channel1


frame = allocate(shape=(frame_in_h, frame_in_w,3),dtype=np.uint8,cacheable=True)

while True: 
    ret, frame_cap = cap.read()
    # Display webcam image via HDMI Out
    if(ret): 
        b, g, r = cv2.split(frame_cap) 
        frame_cap = cv2.merge([g,b,r]) 
        frame[:] = frame_cap[:] 
        vdma.writechannel.writeframe(frame)
        # print("writing frame")
        if(buttons[0].read()): 
            cap.release() 
            vdma.writechannel.stop()
            break
    else:
        print("Failed to read from camera.") 
        cap.release() 
        vdma.writechannel.stop()
        break

print("stop successfully")

# for debug
#cap.release()
#vdma.writechannel.stop()


[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


攝影機成功開啟
stop successfully
